### 304. Range Sum Query 2D - Immutable

#### 🔍 Problem Summary
We are given a 2D matrix and need to efficiently calculate the sum of elements within a rectangular subregion defined by `(row1, col1)` (top-left) and `(row2, col2)` (bottom-right). Multiple queries are expected, so precomputation is essential for speed.

#### 💡 Approach
We use a **2D prefix sum matrix** with an extra row and column of padding (`+1`) to simplify the sumRegion calculations and avoid boundary checks.

1. **Preprocessing (`__init__`)**:
   - Create a new matrix `prefix_sum` of size `(rows + 1) x (cols + 1)` initialized with zeros.
   - For each cell in the original matrix, fill in `prefix_sum[row + 1][col + 1]` using the formula:
     ```
     prefix_sum[r+1][c+1] = matrix[r][c] 
                            + prefix_sum[r][c+1]
                            + prefix_sum[r+1][c]
                            - prefix_sum[r][c]
     ```
   - This ensures `prefix_sum[r+1][c+1]` stores the total sum from `(0,0)` to `(r,c)`.

2. **Querying (`sumRegion`)**:
   - Return the sum of the subrectangle using the inclusion-exclusion principle:
     ```
     sum = total - above - left + top-left overlap
     ```

This results in **O(mn)** time to initialize and **O(1)** time for each query.

In [2]:
class NumMatrix:

    def __init__(self, matrix: list[list[int]]):
        # Number of rows and columns in the original matrix
        rows = len(matrix)
        columns = len(matrix[0])

        # Create a matrix that's 1 row and 1 column larger than the original
        # All values initialized to 0 to simplify prefix sum calculations
        self.prefix_sum = [[0] * (columns + 1) for _ in range(rows + 1)]

        # Fill in the prefix sum matrix
        for row in range(rows):
            for column in range(columns):
                # Each cell stores the sum from (0,0) to (row, column)
                # Adjusted by +1 offset due to the extra row/column at index 0
                self.prefix_sum[row + 1][column + 1] = (
                    matrix[row][column] +                     # current value
                    self.prefix_sum[row][column + 1] +        # sum above
                    self.prefix_sum[row + 1][column] -        # sum left
                    self.prefix_sum[row][column]              # remove double-counted top-left
                )

    def sumRegion(self, row1: int, col1: int, row2: int, col2: int) -> int:
        # (row1, col1): top-left corner of the query rectangle
        # (row2, col2): bottom-right corner

        # The +1 offset adjusts for the padded row and column in prefix_sum
        return (
            self.prefix_sum[row2 + 1][col2 + 1] -  # total area
            self.prefix_sum[row1][col2 + 1] -      # subtract area above
            self.prefix_sum[row2 + 1][col1] +      # subtract area to the left
            self.prefix_sum[row1][col1]            # add back overlap (subtracted twice)
        )

### 🧠 Key Concepts Recap

- **2D Prefix Sum (Integral Image):**
  - Each cell stores the total sum from `(0,0)` to that cell's coordinate.
  - Used for constant-time region sum queries after linear-time preprocessing.

- **Matrix Padding (+1 row/column):**
  - Prevents out-of-bound errors.
  - Simplifies logic for cells on the matrix border.

- **Inclusion-Exclusion Formula for 2D Region:**